In [ ]:
!pip install pymorphy3
!pip install transformers

In [2]:
import os # работа с папкой и файлами
import re # регулярные выражения, доп вариант к очистке
import string # работа со строкой
import math
import operator
import random
import time
import json
import datetime
import csv
import requests
import joblib

import pandas as pd
import numpy as np

from wordcloud import WordCloud, ImageColorGenerator # визуальное отображение
import pymorphy3 # работа с русским языком, pymorphy3

import matplotlib.pyplot as plt # визуальное отображение
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split # Деление выборки на тестовые и тренировочные данные
from sklearn.neighbors import NearestNeighbors

# python -m pip install transformers
from transformers import BertTokenizer # токенизатор BERT
from transformers import BertModel
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import DistilBertTokenizerFast
from transformers import BertForQuestionAnswering
from transformers import DistilBertForQuestionAnswering

import nltk # работа с пакетами языков
from nltk import word_tokenize, ngrams # токенизация и деление на n граммы
from nltk.corpus import stopwords # стопслова, extend

# python -m pip install tensorflow
# python -m pip install torch torchvision torchaudio
import tensorflow
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.metrics import accuracy_score, roc_curve, auc, precision_score, recall_score, f1_score # Критерий качества, точности

# скачаем словарь стоп-слов
nltk.download("stopwords")

# скачиваем модель, которая будет делить на предложения
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sorokin.m.e\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sorokin.m.e\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# Просмотр файлов в директории
print(os.listdir("archive"))

['dev-v1.1.json', 'train-v1.1.json']


In [95]:
# Парсер файлов .json в DataFrame
def convert(f, path=['data', 'paragraphs', 'qas', 'answers']):
  file = json.loads(open(f).read())
  m = pd.json_normalize(file, path[:-1])
  r = pd.json_normalize(file, path[:-2])
  index = np.repeat(r['context'].values, r.qas.str.len())
  m['context'] = index
  data = m[['id', 'question', 'context', 'answers']].set_index('id').reset_index()
  data['context_id'] = data['context'].factorize()[0]
  return data

# Тренировочные данные
train = convert('archive/train-v1.1.json')
train

,id,question,context,answers,context_id
0,5733be284776f41900661182,To whom did the Virgin Mary allegedly appear i...,"Architecturally, the school has a Catholic cha...","[{'answer_start': 515, 'text': 'Saint Bernadet...",0
1,5733be284776f4190066117f,What is in front of the Notre Dame Main Building?,"Architecturally, the school has a Catholic cha...","[{'answer_start': 188, 'text': 'a copper statu...",0
2,5733be284776f41900661180,The Basilica of the Sacred heart at Notre Dame...,"Architecturally, the school has a Catholic cha...","[{'answer_start': 279, 'text': 'the Main Build...",0
3,5733be284776f41900661181,What is the Grotto at Notre Dame?,"Architecturally, the school has a Catholic cha...","[{'answer_start': 381, 'text': 'a Marian place...",0
4,5733be284776f4190066117e,What sits on top of the Main Building at Notre...,"Architecturally, the school has a Catholic cha...","[{'answer_start': 92, 'text': 'a golden statue...",0
...,...,...,...,...,...
87594,5735d259012e2f140011a09d,In what US state did Kathmandu first establish...,"Kathmandu Metropolitan City (KMC), in order to...","[{'answer_start': 229, 'text': 'Oregon'}]",18890
87595,5735d259012e2f140011a09e,What was Yangon previously known as?,"Kathmandu Metropolitan City (KMC), in order to...","[{'answer_start': 414, 'text': 'Rangoon'}]",18890
87596,5735d259012e2f140011a09f,With what Belorussian city does Kathmandu have...,"Kathmandu Metropolitan City (KMC), in order to...","[{'answer_start': 476, 'text': 'Minsk'}]",18890
87597,5735d259012e2f140011a0a0,In what year did Kathmandu create its initial ...,"Kathmandu Metropolitan City (KMC), in order to...","[{'answer_start': 199, 'text': '1975'}]",18890


In [96]:
# Валидационные данные
valid = convert('archive/dev-v1.1.json')
valid

,id,question,context,answers,context_id
0,56be4db0acb8001400a502ec,Which NFL team represented the AFC at Super Bo...,Super Bowl 50 was an American football game to...,"[{'answer_start': 177, 'text': 'Denver Broncos...",0
1,56be4db0acb8001400a502ed,Which NFL team represented the NFC at Super Bo...,Super Bowl 50 was an American football game to...,"[{'answer_start': 249, 'text': 'Carolina Panth...",0
2,56be4db0acb8001400a502ee,Where did Super Bowl 50 take place?,Super Bowl 50 was an American football game to...,"[{'answer_start': 403, 'text': 'Santa Clara, C...",0
3,56be4db0acb8001400a502ef,Which NFL team won Super Bowl 50?,Super Bowl 50 was an American football game to...,"[{'answer_start': 177, 'text': 'Denver Broncos...",0
4,56be4db0acb8001400a502f0,What color was used to emphasize the 50th anni...,Super Bowl 50 was an American football game to...,"[{'answer_start': 488, 'text': 'gold'}, {'answ...",0
...,...,...,...,...,...
10565,5737aafd1c456719005744fb,What is the metric term less used than the New...,"The pound-force has a metric counterpart, less...","[{'answer_start': 82, 'text': 'kilogram-force'...",2066
10566,5737aafd1c456719005744fc,What is the kilogram-force sometimes reffered ...,"The pound-force has a metric counterpart, less...","[{'answer_start': 114, 'text': 'kilopond'}, {'...",2066
10567,5737aafd1c456719005744fd,What is a very seldom used unit of mass in the...,"The pound-force has a metric counterpart, less...","[{'answer_start': 274, 'text': 'slug'}, {'answ...",2066
10568,5737aafd1c456719005744fe,What seldom used term of a unit of force equal...,"The pound-force has a metric counterpart, less...","[{'answer_start': 712, 'text': 'kip'}, {'answe...",2066


In [97]:
# Размер тренировочных данных
print(f'Размер тренировочных данных: {train.shape}')

# Количество уникальных контекстов в наборе тренировочных данных
print(f'Количество уникальных контекстов в наборе тренировочных данных: {train["context_id"].unique().size}')

# Список уникальных контекстов в наборе тренировочных данных
train_unique = train[['context', 'context_id']].drop_duplicates().reset_index(drop=True)
train_unique

Размер тренировочных данных: (87599, 5)
Количество уникальных контекстов в наборе тренировочных данных: 18891


,context,context_id
0,"Architecturally, the school has a Catholic cha...",0
1,"As at most other universities, Notre Dame's st...",1
2,The university is the major seat of the Congre...,2
3,The College of Engineering was established in ...,3
4,All of Notre Dame's undergraduate students are...,4
...,...,...
18886,"Institute of Medicine, the central college of ...",18886
18887,Football and Cricket are the most popular spor...,18887
18888,The total length of roads in Nepal is recorded...,18888
18889,The main international airport serving Kathman...,18889


In [98]:
# Размер валидационных данных
print(f'Размер валидационных данных: {valid.shape}')

# Количество уникальных контекстов в наборе валидационных данных
print(f'Количество уникальных контекстов в наборе валидационных данных: {valid["context_id"].unique().size}')

# Список уникальных контекстов в наборе валидационных данных
valid_unique = valid[['context', 'context_id']].drop_duplicates().reset_index(drop=True)
valid_unique

Размер валидационных данных: (10570, 5)
Количество уникальных контекстов в наборе валидационных данных: 2067


,context,context_id
0,Super Bowl 50 was an American football game to...,0
1,The Panthers finished the regular season with ...,1
2,The Broncos took an early lead in Super Bowl 5...,2
3,"CBS broadcast Super Bowl 50 in the U.S., and c...",3
4,"In early 2012, NFL Commissioner Roger Goodell ...",4
...,...,...
2062,"where is the mass of the object, is the velo...",2062
2063,A conservative force that acts on a closed sys...,2063
2064,"For certain physical scenarios, it is impossib...",2064
2065,The connection between macroscopic nonconserva...,2065


In [99]:
# Контексты тренировочных данных
train_contexts = train['context'].to_numpy().tolist()
train_contexts

['Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building

In [100]:
# Контексты валидационных данных
valid_contexts = valid['context'].to_numpy().tolist()
valid_contexts

['Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.',
 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Foo

In [101]:
# Вопросы тренировочных данных
train_questions = train['question'].to_numpy().tolist()
train_questions

['To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'What is in front of the Notre Dame Main Building?',
 'The Basilica of the Sacred heart at Notre Dame is beside to which structure?',
 'What is the Grotto at Notre Dame?',
 'What sits on top of the Main Building at Notre Dame?',
 'When did the Scholastic Magazine of Notre dame begin publishing?',
 "How often is Notre Dame's the Juggler published?",
 'What is the daily student paper at Notre Dame called?',
 'How many student news papers are found at Notre Dame?',
 'In what year did the student paper Common Sense begin publication at Notre Dame?',
 'Where is the headquarters of the Congregation of the Holy Cross?',
 'What is the primary seminary of the Congregation of the Holy Cross?',
 'What is the oldest structure at Notre Dame?',
 'What individuals live at Fatima House at Notre Dame?',
 'Which prize did Frederick Buechner create?',
 'How many BS level degrees are offered in the College of Engineering at Notre

In [102]:
# Вопросы валидационных данных
valid_questions = valid['question'].to_numpy().tolist()
valid_questions

['Which NFL team represented the AFC at Super Bowl 50?',
 'Which NFL team represented the NFC at Super Bowl 50?',
 'Where did Super Bowl 50 take place?',
 'Which NFL team won Super Bowl 50?',
 'What color was used to emphasize the 50th anniversary of the Super Bowl?',
 'What was the theme of Super Bowl 50?',
 'What day was the game played on?',
 'What is the AFC short for?',
 'What was the theme of Super Bowl 50?',
 'What does AFC stand for?',
 'What day was the Super Bowl played on?',
 'Who won Super Bowl 50?',
 'What venue did Super Bowl 50 take place in?',
 'What city did Super Bowl 50 take place in?',
 'If Roman numerals were used, what would Super Bowl 50 have been called?',
 'Super Bowl 50 decided the NFL champion for what season?',
 'What year did the Denver Broncos secure a Super Bowl title for the third time?',
 'What city did Super Bowl 50 take place in?',
 'What stadium did Super Bowl 50 take place in?',
 'What was the final score of Super Bowl 50? ',
 'What month, day and y

In [103]:
# Ответы тренировочных данных
train_answers = []
for i, value in enumerate(train['answers'].to_numpy().tolist()):
  train_answers.append(value[0]['text'])
train_answers

['Saint Bernadette Soubirous',
 'a copper statue of Christ',
 'the Main Building',
 'a Marian place of prayer and reflection',
 'a golden statue of the Virgin Mary',
 'September 1876',
 'twice',
 'The Observer',
 'three',
 '1987',
 'Rome',
 'Moreau Seminary',
 'Old College',
 'Retired priests and brothers',
 'Buechner Prize for Preaching',
 'eight',
 '1920',
 'the College of Science',
 'five',
 'the 1870s',
 'Learning Resource Center',
 'five',
 'The First Year of Studies program',
 'U.S. News & World Report',
 '1924',
 'Master of Divinity',
 'Alliance for Catholic Education',
 '1854',
 'Department of Pre-Professional Studies',
 'Joan B. Kroc Institute for International Peace Studies',
 'President Emeritus of the University of Notre Dame',
 '1986',
 'Ray Kroc',
 "McDonald's",
 '14',
 'Theodore M. Hesburgh Library',
 '1963',
 'Millard Sheets',
 'Touchdown Jesus',
 '3,577',
 '19.7%',
 'the top 10 to 15 in the nation',
 '39.1%',
 'more than 750 miles',
 '18th overall',
 '8th',
 '1st overa

In [104]:
# Ответы валидационных данных
valid_answers = []
for i, value in enumerate(valid['answers'].to_numpy().tolist()):
  valid_answers.append(value[0]['text'])
valid_answers

['Denver Broncos',
 'Carolina Panthers',
 'Santa Clara, California',
 'Denver Broncos',
 'gold',
 '"golden anniversary"',
 'February 7, 2016',
 'American Football Conference',
 '"golden anniversary"',
 'American Football Conference',
 'February 7, 2016',
 'Denver Broncos',
 "Levi's Stadium",
 'Santa Clara',
 'Super Bowl L',
 '2015',
 '2015',
 'Santa Clara',
 "Levi's Stadium",
 '24–10',
 'February 7, 2016',
 '2015',
 'Denver Broncos',
 'Carolina Panthers',
 'Denver Broncos',
 '2015',
 'Denver Broncos',
 'Santa Clara, California.',
 'Super Bowl',
 'Denver Broncos',
 'Cam Newton',
 '8',
 '1995',
 'Arizona Cardinals',
 'New England Patriots',
 'Arizona Cardinals',
 'New England Patriots',
 'New England Patriots',
 'four',
 'Cam Newton',
 '15–1',
 'Cam Newton',
 '12–4',
 '4',
 'New England Patriots',
 'Cam Newton',
 'Arizona Cardinals',
 '2',
 'New England Patriots',
 'Cam Newton',
 'New England Patriots',
 'Arizona Cardinals',
 'Cam Newton',
 'Arizona Cardinals',
 '1995.',
 'Von Miller',
 

In [105]:
# Общие тренировочные данные
train_merge = pd.merge(pd.DataFrame(train_contexts, columns=['contexts']), pd.DataFrame(train_questions, columns=['questions']), left_index=True, right_index=True)
train_data = pd.merge(train_merge, pd.DataFrame(train_answers, columns=['answers']), left_index=True, right_index=True)
train_data

,contexts,questions,answers
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,Saint Bernadette Soubirous
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,a copper statue of Christ
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,the Main Building
3,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,a Marian place of prayer and reflection
4,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,a golden statue of the Virgin Mary
...,...,...,...
87594,"Kathmandu Metropolitan City (KMC), in order to...",In what US state did Kathmandu first establish...,Oregon
87595,"Kathmandu Metropolitan City (KMC), in order to...",What was Yangon previously known as?,Rangoon
87596,"Kathmandu Metropolitan City (KMC), in order to...",With what Belorussian city does Kathmandu have...,Minsk
87597,"Kathmandu Metropolitan City (KMC), in order to...",In what year did Kathmandu create its initial ...,1975


In [106]:
# Общие валидационные данные
valid_merge = pd.merge(pd.DataFrame(valid_contexts, columns=['contexts']), pd.DataFrame(valid_questions, columns=['questions']), left_index=True, right_index=True)
valid_data = pd.merge(valid_merge, pd.DataFrame(valid_answers, columns=['answers']), left_index=True, right_index=True)
valid_data

,contexts,questions,answers
0,Super Bowl 50 was an American football game to...,Which NFL team represented the AFC at Super Bo...,Denver Broncos
1,Super Bowl 50 was an American football game to...,Which NFL team represented the NFC at Super Bo...,Carolina Panthers
2,Super Bowl 50 was an American football game to...,Where did Super Bowl 50 take place?,"Santa Clara, California"
3,Super Bowl 50 was an American football game to...,Which NFL team won Super Bowl 50?,Denver Broncos
4,Super Bowl 50 was an American football game to...,What color was used to emphasize the 50th anni...,gold
...,...,...,...
10565,"The pound-force has a metric counterpart, less...",What is the metric term less used than the New...,kilogram-force
10566,"The pound-force has a metric counterpart, less...",What is the kilogram-force sometimes reffered ...,kilopond
10567,"The pound-force has a metric counterpart, less...",What is a very seldom used unit of mass in the...,slug
10568,"The pound-force has a metric counterpart, less...",What seldom used term of a unit of force equal...,kip


In [109]:
# Загрузка предтренированной модели BERT
# model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [110]:
# Загрузка предтренированного токенайзера BERT
# tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [111]:
train_encodings = tokenizer(train_data['contexts'].values.tolist(), train_data['questions'].values.tolist(), truncation=True, padding=True)
validation_encodings = tokenizer(valid_data['contexts'].values.tolist(), valid_data['questions'].values.tolist(), truncation=True, padding=True)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [116]:
def question_answer(question, text):
  #tokenize question and text in ids as a pair
  input_ids = tokenizer.encode(question, text)
  #string version of tokenized ids
  tokens = tokenizer.convert_ids_to_tokens(input_ids)
  #segment IDs
  #first occurence of [SEP] token
  sep_idx = input_ids.index(tokenizer.sep_token_id)
  #number of tokens in segment A - question
  num_seg_a = sep_idx+1
  #number of tokens in segment B - text
  num_seg_b = len(input_ids) - num_seg_a
  #list of 0s and 1s
  segment_ids = [0]*num_seg_a + [1]*num_seg_b
  assert len(segment_ids) == len(input_ids)
  #model output using input_ids and segment_ids
  output = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))
  #reconstructing the answer
  answer_start = torch.argmax(output.start_logits)
  answer_end = torch.argmax(output.end_logits)
  if answer_end >= answer_start:
    answer = tokens[answer_start]
    for i in range(answer_start+1, answer_end+1):
      if tokens[i][0:2] == "##":
        answer = ""
      else:
        answer += " " + tokens[i]
  if answer.startswith("[CLS]"):
    answer = "Unable to find the answer to your question."
  print("Context:\n{}".format(text.capitalize()))
  print("\nQuestion:\n{}".format(question.capitalize()))
  print("\nAnswer:\n{}".format(answer.capitalize()))

question_answer(train_data['questions'].values.tolist()[0], train_data['contexts'].values.tolist()[0])

Context:
Architecturally, the school has a catholic character. atop the main building's gold dome is a golden statue of the virgin mary. immediately in front of the main building and facing it, is a copper statue of christ with arms upraised with the legend "venite ad me omnes". next to the main building is the basilica of the sacred heart. immediately behind the basilica is the grotto, a marian place of prayer and reflection. it is a replica of the grotto at lourdes, france where the virgin mary reputedly appeared to saint bernadette soubirous in 1858. at the end of the main drive (and in a direct line that connects through 3 statues and the gold dome), is a simple, modern stone statue of mary.

Question:
To whom did the virgin mary allegedly appear in 1858 in lourdes france?

Answer:
 with the legend "


In [115]:
train_data['answers'].values.tolist()[0]

'Saint Bernadette Soubirous'

In [117]:
embedding = TfidfVectorizer(
  encoding = 'utf-8',
  lowercase = True,
  analyzer = 'word',
  stop_words = 'english',
  binary = True,
  max_features = 300
)

retriever = NearestNeighbors(
  n_neighbors = 1,
  metric = 'cosine'
)

X_train = embedding.fit_transform(train_unique['context'])
retriever.fit(X_train, train_unique['context_id'])

question = 'What are the tourist hostspots in Spain?'
print(embedding.inverse_transform(embedding.transform([question])))

X_question = embedding.transform([question])
context_id = retriever.kneighbors(X_question, return_distance=False)[0][0]
selected_document = train_unique.iloc[context_id]['context']
print(embedding.inverse_transform(embedding.transform([selected_document])))

X_train = embedding.transform(train['question'])
y_test = train['context_id']
y_pred = retriever.kneighbors(X_train, return_distance=False)
print(y_pred)

[array([], dtype='<U13')]
[array(['school', 'place', 'modern', 'main', 'line', 'france', 'end',
       'building'], dtype='<U13')]
